# Trabalhando com dados de texto

## **Tokenização de texto**

O texto que iremos tokenizar para o treinamento LLM é um conto de Edith Wharton chamado The Verdict

In [ ]:
!wget -O the-verdict.txt "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

--2024-03-05 16:37:12--  https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20479 (20K) [text/plain]
Saving to: ‘the-verdict.txt’

the-verdict.txt     100%[===================>]  20.00K  --.-KB/s    in 0s      

2024-03-05 16:37:12 (94.6 MB/s) - ‘the-verdict.txt’ saved [20479/20479]



In [ ]:
# Lendo um conto como exemplo de texto em python
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read() # lê o arquivo de texto e o armazena na var raw_text

print("Total number of character: ", len(raw_text))
print(raw_text[:99])

Total number of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


GOAL: Transformar esses 20479 caracteres em palavras individuas e caracteres individuais

In [ ]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [ ]:
# separando pontos e virgulas
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [ ]:
result = [item.strip() for item in result if item.strip()] # remove qualquer espaço em branco, se o item tiver espaço em branco
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [ ]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


basicamente o que fizemos:

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-3/Figures/02__image009.png">

In [ ]:
# aplicando em todo o conto de Edith Wharton:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4649


In [ ]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## **Convertendo tokens em IDs de token**

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-3/Figures/02__image011.png">

In [ ]:
# criando uma lista de tokens exclusivos
all_words = sorted(list(set(preprocessed))) # sorted = organiza por ordem alfabetica, set = conjunto, remove as duplicatas
vocab_size = len(all_words)
print(vocab_size)

1159


In [ ]:
# criando um vocabulario:
vocab = {token:integer for integer, token in enumerate(all_words)}
for i,item in enumerate(vocab.items()):
  print(item)
  if i > 50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)
('Her', 51)


In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

Vamos instanciar um novo objeto tokenizer

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]


In [ ]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

Ok ele funcionou bem, vamos agora aplica-lo a um novo exemplo de texto que não está contido no conjunto de treinamento:


In [ ]:
# esse código dará erro:
#text = "Hello, do you like tea?"
#tokenizer.encode(text)

O problema é que a palavra "hello" não foi usada no The Verdict conto . Portanto, não está contido no vocabulário. Isto destaca a necessidade de considerar conjuntos de formação grandes e diversificados para ampliar o vocabulário ao trabalhar em LLMs.

## **Adicionando tokens de contexto especiais**

UNK E ENDOFTEXT

Vamos agora modificar o vocabulário para incluir esses dois tokens especiais, <unk>e <|endoftext|>, adicionando-as à lista de todas as palavras exclusivas que criamos na seção anterior:

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

print(len(vocab.items()))

1161


In [ ]:
# vamos imprimir as ultimas 5 entradas do vocabulario atualizado para conferir
for i,item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)


In [ ]:
# Um tokenizador de texto simples que lida com palavras desconhecidas

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
# vamos testar essa bosta agora
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1160, 5, 362, 1155, 642, 1000, 10, 1159, 57, 1013, 981, 1009, 738, 1013, 1160, 7]


1159 -> <|endoftext|>

1160 -> <|UNK|>

In [ ]:
# destokenização QUE PALAVRA DIFICIL BRO
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## Byte pair encoding

In [ ]:
# algoritmo de BPE baseado em rust
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00


In [ ]:
import importlib
import tiktoken
print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.6.0


In [ ]:
# Instanciando o tokenizer BPE do tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

Esse tokenizer é semelhante ao `SimpleTokenizerV2`

In [ ]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [ ]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [ ]:
# Testando
texto = "Akwirw ier"
test = tokenizer.encode(texto)
print(test)

[33901, 86, 343, 86, 220, 959]


In [ ]:
c_decode = tokenizer.decode(test)
c_decode

'Akwirw ier'

## **Amostragem de dados com janela deslizante**

Dada uma amostra de texto, extraia blocos de entrada como subamostras que servem como entrada para o LLM, e a tarefa de previsão do LLM durante o treinamento é prever a próxima palavra que segue o bloco de entrada. Durante o treinamento, mascaramos todas as palavras que ultrapassam o alvo. Observe que o texto mostrado nesta figura passaria por tokenização antes que o LLM pudesse processá-lo; no entanto, esta figura omite a etapa de tokenização para maior clareza.


<img src="https://drek4537l1klr.cloudfront.net/raschka/v-3/Figures/02__image023.png">


In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


5145 tokens após a aplicação do tokenizer BPE

In [ ]:
#oi

In [ ]:
# rmeovendo os 50 primeiros tokens pq sim
enc_sample = enc_text[50:]


Uma das maneiras mais fáceis e intuitivas de criar pares de entrada-alvo para a tarefa de previsão da próxima palavra é criar duas variáveis, xe y, onde xcontém os tokens de entrada e ycontém os alvos, que são as entradas **deslocadas** em 1:

In [ ]:
# O tamanho do contexto determina quantos tokens estão incluídos na entrada

context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")


x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


Para fins de ilustração, vamos repetir o código anterior, mas converter os IDs dos tokens em texto:

In [ ]:
for i in range(1, context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(tokenizer.decode(context), "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


Há apenas mais uma tarefa antes de podermos transformar os tokens em embeddings, como mencionamos no início deste capítulo: implementar um carregador de dados eficiente que itere sobre o conjunto de dados de entrada e retorne as entradas e os alvos como tensores PyTorch.


Para implementar carregadores de dados eficientes, coletamos as entradas em um tensor, x, onde cada linha representa um contexto de entrada. Um segundo tensor, y, contém os alvos de previsão correspondentes (próximas palavras), que são criados mudando a entrada em uma posição.

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-3/Figures/02__image025.png">

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) # tokenize o texto todo

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self): # retorna a quantidade total de linhas no conjunto de dados
        return len(self.input_ids)

    def __getitem__(self, idx): # retorna apenas uma linha no conjunto de dados.
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
# Um carregador de dados (dataloader) para gerar lotes com pares entrada-alvo
def create_dataloader(txt, batch_size=4,
        max_length=256, stride=128, shuffle=True):
    tokenizer = tiktoken.get_encoding("gpt2") # inicializa o tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) # cria o dataset
    dataloader = DataLoader( # cria o dataloader
        dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader


Vamos testar o dataloadercom tamanho de lote 1 para um LLM com tamanho de contexto 4 para desenvolver uma intuição de como o GPTDatasetV1classe da listagem 2.5 e a create_dataloaderfunção da listagem 2.6 funcionam juntas:

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


Para ilustrar o significado de stride=1, vamos buscar outro lote deste conjunto de dados:

stride tem a ver com deslocamento

In [ ]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


Se compararmos o primeiro com o segundo batch (lote), podemos ver que os IDs de token do segundo lote saão **deslocados** em uma posição em comparação com o primeiro lote (por exemplo, o segundo ID na entrada do primeiro lote é 367, que é o primeiro ID de entrada do segundo lote). A configuração `Stride`  determina o número de posições que as entradas mudam entre lotes, emulando uma abordagem de janela deslizante.

In [ ]:
third_batch = next(data_iter)
print(third_batch)

[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]


Ao criar vários lotes a partir do conjunto de dados de entrada, deslizamos uma janela de entrada pelo texto. Se o passo for definido como 1, deslocaremos a janela de entrada em 1 posição ao criar o próximo lote. Se definirmos o passo igual ao tamanho da janela de entrada, podemos evitar sobreposições entre os lotes.

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-3/Figures/02__image027.png">

vamos dar uma breve olhada em como podemos usar o carregador de dados para fazer amostras com um tamanho de lote maior que 1:

In [ ]:
dataloader = create_dataloader(raw_text, batch_size=8, max_length=4, stride=5)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  607,   599,  6321,   287],
        [  465, 11441, 48740,    11],
        [  520,  5493,  2241,   318],
        [  287,  1936,  2431,   438],
        [   11,   465, 10904,  4252],
        [ 1576,   284,   766,   465],
        [15910,    13,   887,   484],
        [  262, 37090,   257,   845]])

Targets:
 tensor([[  599,  6321,   287,   262],
        [11441, 48740,    11,   345],
        [ 5493,  2241,   318,   262],
        [ 1936,  2431,   438,   392],
        [  465, 10904,  4252,  6236],
        [  284,   766,   465,  5986],
        [   13,   887,   484,  4054],
        [37090,   257,   845, 22665]])


## **Criando incorporações(embedding) de token**

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-3/Figures/02__image029.png">


In [ ]:
# suponha que temos os seguintes quatro tokens de entrada com IDs 5,1,3 e 2
input_ids = torch.tensor([5,1,3,4])

Para fins de simplicidade e ilustração, suponha que temos um pequeno vocabulário de apenas 6 palavras (em vez das 50.257 palavras no vocabulário do tokenizer BPE) e queremos criar embeddings de tamanho 3 (no GPT-3, o tamanho de incorporação tem 12.288 dimensões):

In [ ]:
vocab_size = 6
output_dim = 3

In [ ]:
# Instanciando uma camada de Incorporação (embedding) though pytorch
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


A instrução print no exemplo de código anterior imprime a matriz de peso subjacente da camada de incorporação

In [ ]:
# Vamos aplicar a um ID de token para obter o vetor de embedding
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


Se compararmos o vetor de incorporação do token ID 3 com a matriz de incorporação anterior, veremos que ele é idêntico à 4ª linha (Python começa com um índice zero, portanto é a linha correspondente ao índice 3). Em outras palavras, a camada de incorporação é essencialmente uma operação de pesquisa que recupera linhas da matriz de pesos da camada de incorporação por meio de um ID de token.

In [ ]:
# Vamos agora aplicar a todos os IDs de entradas que definimos anteriormente
print(embedding_layer(input_ids))

tensor([[-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315]], grad_fn=<EmbeddingBackward0>)


As camadas de incorporação realizam uma operação de pesquisa, recuperando o vetor de incorporação correspondente ao ID do token da matriz de pesos da camada de incorporação. Por exemplo, o vetor de incorporação do token ID 5 é a sexta linha da matriz de peso da camada de incorporação (é a sexta em vez da quinta linha porque o Python começa a contar em 0).

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-3/Figures/02__image031.png">

## **Codificando posições de palavras**

Embeddings posicionais são adicionados ao vetor de incorporação de token para criar os embeddings de entrada para um LLM. Os vetores posicionais têm a mesma dimensão que os embeddings de tokens originais. Os embeddings de token são mostrados com valor 1 para simplificar.

<img src="https://drek4537l1klr.cloudfront.net/raschka/v-4/Figures/02__image035.png">

In [ ]:
output_dim = 256
vocab_size = 50257

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
# basicamente essa bosta terá 50257 linhas com 256 colunas cada

In [ ]:
# exemplo de uma linha na camada embedding:
token_embedding_layer.weight[0]

tensor([-2.1338e+00,  1.0524e+00, -3.8848e-01, -9.3435e-01, -4.9914e-01,
        -1.0867e+00,  9.6242e-01,  2.4921e-01,  6.2662e-01, -1.7549e-01,
         9.8284e-02, -9.3507e-02,  2.6621e-01, -5.8504e-01, -3.4304e-01,
        -6.8215e-01, -1.4779e+00,  1.1331e+00, -1.2203e+00,  1.3139e+00,
         1.0533e+00,  1.3881e-01, -2.0444e-01, -2.2685e+00, -2.8084e-01,
         7.6968e-01, -6.5956e-01, -7.9793e-01,  1.8383e-01,  2.2935e-01,
         6.1774e-01, -2.8758e-01, -2.5873e-01, -1.0826e+00, -4.4382e-02,
         1.6236e+00, -2.3229e+00,  1.0878e+00, -6.3545e-02, -4.4864e-01,
        -9.4872e-01, -7.6507e-02, -1.5264e-01,  1.1674e-01,  4.4026e-01,
        -1.4465e+00, -5.5808e-01, -5.1696e-02,  1.0042e+00,  8.2723e-01,
        -3.9481e-01,  4.8923e-01, -2.1681e-01, -1.7472e+00,  1.7228e+00,
         7.7381e-01,  9.0315e-01, -7.2184e-01, -5.9508e-01, -7.1122e-01,
         6.2296e-01, -1.3729e+00, -1.2806e-01, -1.2838e+00, -2.0915e+00,
         9.6285e-01, -3.1861e-02, -4.7896e-01,  7.6

In [ ]:
max_length = 4

dataloader = create_dataloader(raw_text, batch_size=8, max_length=max_length, stride=5, shuffle=False)
data_iter = iter(dataloader)

inputs, targets = next(data_iter)
print("Token IDs: \n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 3619,   402,   271, 10899],
        [  257,  7026, 15632,   438],
        [  257,   922,  5891,  1576],
        [  568,   340,   373,   645],
        [ 5975,   284,   502,   284],
        [  326,    11,   287,   262],
        [  286,   465, 13476,    11]])

Inputs shape:
 torch.Size([8, 4])


In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
block_size = max_length
pos_embedding_layer = torch.nn.Embedding(block_size, output_dim)
pos_embedding = pos_embedding_layer(torch.arange(block_size))
print(pos_embedding.shape)

torch.Size([4, 256])


In [ ]:
input_embeddings = token_embeddings + pos_embedding
print(input_embeddings.shape)

torch.Size([8, 4, 256])


<img src="https://drek4537l1klr.cloudfront.net/raschka/v-4/Figures/02__image037.png">